# March Madness 2024 Code
### Team: Taylor Last, Ted Woodsides, Jake Hopkins

### Notes
- **Please add docstrings or commments to code so we all know what's going on**
- I can't find how to all work on the same notebook so we might have to copy code and all work on separate pieces
- **Check create_submission and simulate functions to see how to get a bracket in the form the competition asks for**
### Things to keep in mind
- **Days are standardized already**: Dayzero tells you the date corresponding to DayNum=0 during that season. All game dates have been aligned upon a common scale so that (each year) the Monday championship game of the men's tournament is on DayNum=154. Working backward, the men's national semifinals are always on DayNum=152, the "play-in" games are on days 134-135, men's Selection Sunday is on day 132, the final day of the regular season is also day 132, and so on. 
- **Special note about "Season" numbers**: the college basketball season lasts from early November until the national championship tournament that starts in the middle of March. For instance, this year the first regular season games were played in November 2023 and the national championship games will be played in April 2024. Because a basketball season spans two calendar years like this, it can be confusing to refer to the year of the season. By convention, when we identify a particular season, we will reference the year that the season ends in, not the year that it starts in. So for instance, the current season will be identified in our data as the 2024 season, not the 2023 season or the 2023-24 season or the 2023-2024 season, though you may see any of these in everyday use outside of our data.

### Questions to consider
- Do we want to train the model by making prediction for all games (regular season and post), then only make predictions for tournament games, or do we want to only train on tournament games?

### Featues I'd like to add
- Quantify guard play
- Free throw percentage
- Pace
- Time of possession

In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from typing import List
from tqdm import tqdm
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost

import networkx
from networkx.algorithms.traversal.depth_first_search import dfs_edges

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Refer to this if you need to look up documentation
print(f"Pandas Version: {pd.__version__}")

Pandas Version: 2.2.0


In [3]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
data = dict()
for dirname, _, filenames in os.walk('../kaggle/input'):
    for filename in filenames:
        table_name = filename.split('.')[0]
        table_path = os.path.join(dirname, filename)
        try:
            data[table_name] = pd.read_csv(table_path)
        except UnicodeDecodeError:
            data[table_name] = pd.read_csv(table_path, encoding='cp1252')
        except Exception as e:
            print(f"Error with {filename}: {e}")

# Split dict of dataframes by gender and other (supplemental) data
mens_data = dict()
womens_data = dict()
supplemental_data = dict()

for k, v in data.items():
    if k.startswith("M"):
        mens_data[k] = v
    elif k.startswith("W"):
        womens_data[k] = v
    else:
        supplemental_data[k] = v
        

In [4]:
def get_season_stats(dataset, detailed=False, post_season=False, year=2024):
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if detailed:
        if post_season:
            df = dataset[f"{gender}NCAATourneyDetailedResults"]
        else:
            df = dataset[f"{gender}RegularSeasonDetailedResults"]
        
    else:
        if post_season:
            df = dataset[f"{gender}NCAATourneyCompactResults"]
        else:
            df = dataset[f"{gender}RegularSeasonCompactResults"]
        
    df = df[df["Season"] == year]
    return df, gender

def compute_margins_of_victory(df):
    df["margin"] = df["WScore"] - df["LScore"]
    
    win_df = df[["WTeamID", "margin"]].rename(columns={"WTeamID": "TeamID"})
    lose_df = df[["LTeamID", "margin"]].rename(columns={"LTeamID": "TeamID"})
    lose_df["margin"] = -lose_df["margin"]

    res = pd.concat([win_df, lose_df], axis=0)
    return res.groupby("TeamID")["margin"].mean()

def join_team_names(df, data, gender="M"):
    """
    df: pd.DataFrame
        dataframe appending teams to
    data: dict[str, pd.DataFrame]
        dictionary of all table names and data
    """
    res = pd.merge(df, data[f"{gender}Teams"][["TeamID", "TeamName"]], on="TeamID")
    return res

def create_srs(df,gender):

    df["margin"] = df["WScore"] - df["LScore"]
    win_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"WTeamID": "team_id", "LTeamID": "opp_id"}
    )
    lose_df = df[["WTeamID", "margin", "LTeamID"]].rename(
        columns={"LTeamID": "team_id", "WTeamID": "opp_id"}
    )
    lose_df["margin"] = -lose_df["margin"]

    teams = pd.concat([win_df, lose_df], axis=0)
    spreads = compute_margins_of_victory(df)
    
    terms = []
    solutions = []

    for team_id in spreads.keys():
        row = []
        opps = list(teams[teams["team_id"] == team_id]["opp_id"])

        for opp_id in spreads.keys():
            if opp_id == team_id:
                # coef for the team itself should be 1
                row.append(1)
            elif opp_id in opps:
                # coef for opponents is 1 over num of opps
                row.append(-1.0/len(opps))
            else:
                # teams not faced get a 0 coef
                row.append(0)
        terms.append(row)

        solutions.append(spreads[team_id])

    solutions, _, _, _ = np.linalg.lstsq(np.array(terms), np.array(solutions), rcond=None)
    
    ratings = list(zip( spreads.keys(), solutions ))
    srs = pd.DataFrame(ratings, columns=['team', 'rating'])
    rankings = srs.sort_values('rating', ascending=False).reset_index()[['team', 'rating']]
    rankings = join_team_names(rankings.rename(columns={"team": "TeamID"}), data, gender=gender)
    return rankings

def get_coach_win_perc(
    dataset: dict,
    regular_season: bool,
    year:int = 2024
) -> pd.DataFrame:
    """
    
    parameters
    ----------
    dataset: dict
        dictionary of datasets to use. it will be
        mens_data or womens_data.
        
    year: int
        year to filter data. it will get coaches stats for everything
        up until this year. (model can't have any look ahead bias). for post
        season games, use a year one less than the year of interest.
        
    returns
    -------
    coaches_stats: pd.DataFrame
        dataframe with count of wins, win percentage, and std dev
        of wins.
    """
    
    # Gets the first letter in dataset
    gender = list(dataset.keys())[0][0]
    
    if regular_season:
        df = dataset[f"{gender}RegularSeasonCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] <= year]
    else:
        df = dataset[f"{gender}NCAATourneyCompactResults"]
        #Filter season up until season of interest
        df = df[df["Season"] < year]
        
    
    
    winning_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "WTeamID"],
        right_on=["Season", "TeamID"]
    )

    winning_coaches_df = winning_coaches_df[
        (winning_coaches_df['DayNum'] >= winning_coaches_df['FirstDayNum']) 
        & (winning_coaches_df['DayNum'] <= winning_coaches_df['LastDayNum'])
    ]
    winning_coaches_df["win"] = 1

    #Make sure the join dind't create dupes
    assert len(winning_coaches_df) == len(df)

    losing_coaches_df = pd.merge(
        df,
        dataset[f"{gender}TeamCoaches"],
        how="left",
        left_on=["Season", "LTeamID"],
        right_on=["Season", "TeamID"]
    )

    losing_coaches_df = losing_coaches_df[
        (losing_coaches_df['DayNum'] >= losing_coaches_df['FirstDayNum']) 
        & (losing_coaches_df['DayNum'] <= losing_coaches_df['LastDayNum'])
    ]
    losing_coaches_df["win"] = 0

    #Make sure the join dind't create dupes
    assert len(losing_coaches_df) == len(df)

    coaches_df = pd.concat(
        [
            losing_coaches_df[["CoachName", "win"]],
            winning_coaches_df[["CoachName", "win"]]
        ],
        axis=0
    )

    coach_stats = (
        coaches_df
        .groupby("CoachName")["win"]
        .describe()
        .sort_values("count", ascending=False)
        [["count", "mean", "std"]]
        .fillna(0)
    )

    return coach_stats
def get_system_ratings(
    mens_dataset, #There are only ratings for men
    systems: List[str],
    year: int=2024,
):
    """
    gets system ratings for each team for specified systems for a specific year.
    
    parameters
    ---------
    mens_dataset: dict
        dictionary of datasets for men
    systems: List[str]
        list of dictionaries we are interested in seeing
    year: int
        year to look for ratings
    moving_average: str
        specifies how to calculate rolling ratings for given systems.
        if None, the system takes the most recent system rating
    
    returns
    -------
    df: pd.DataFrame
        data that reflects ratings for a team
    """
    
    # Filter by season - only take most recent
    df = mens_dataset["MMasseyOrdinals"]
    df = df[df["Season"] == year]
    
    # Filter by system
    df = df[df["SystemName"].isin(systems)]
    
    latest_rank = (
        df
        .sort_values("RankingDayNum")
        .groupby(["TeamID","SystemName"])
        ["OrdinalRank"]
        .last()
        .unstack("SystemName")
        .reset_index().
        rename(columns = {i: i+"_latest" for i in systems})
    )
    
    transformed_df = (
        df
        .sort_values(by="RankingDayNum")
        .groupby(["TeamID", "SystemName"], group_keys=False)
        ["OrdinalRank"]
        .rolling(5) # TODO: Parameterize this (window and moving average method)
        .mean()
        .unstack("SystemName")
        .reset_index()
        .drop("level_1", axis=1)
        .groupby("TeamID")
        [systems]
        .last()
        .reset_index()
        .rename(columns = {i: i+"_rolling" for i in systems})
    )
    
    res = pd.merge(latest_rank, transformed_df, on="TeamID")

    return res

def get_post_season(data, year):
    
    df, gender = get_season_stats(
            data, 
            detailed=False, 
            post_season=True, 
            year=year
    )
    
    # Shuffle teams for positional encoding (model shouldn't have winning teams features as the same)
    df["TeamID"] = np.where(
        np.random.uniform(0,1, size=len(df)) > .5, 
        df["WTeamID"], 
        df["LTeamID"]
    )
    df["team_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["WScore"], 
        df["LScore"]
    )
    df["OppID"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LTeamID"], 
        df["WTeamID"]
    )
    df["opp_score"] = np.where(
        df["TeamID"] == df["WTeamID"], 
        df["LScore"], 
        df["WScore"]
    )
    df = df.drop(
        ["WTeamID", "LTeamID", "WScore", "LScore", "WLoc", "NumOT"],
        axis=1
    )
    
    return df

def get_features(mens_data, year, systems):
    # Season Stats
    df, gender = get_season_stats(
        mens_data, 
        detailed=False, 
        post_season=False, 
        year=year
    )

    # Rating System
    srs = create_srs(df, gender)

    # System Ratings
    system_ratings = get_system_ratings(
        mens_data, 
        systems=systems
    ) #KenPom, Nolan ELO, EPSN BPI

    # Ratings df
    ratings_df = pd.merge(
                srs,
                system_ratings,
                on="TeamID"
    )

    # Coaches postseason win stats
    coaches_postseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=False, 
        year=year
    ).rename(columns={"count": "count_post", "mean": "mean_post", "std": "std_post"})

    # Coaches regular season win stats
    coaches_regseason_win_df = get_coach_win_perc(
        dataset=mens_data, 
        regular_season=True, 
        year=year
    ).rename(columns={"count": "count_reg", "mean": "mean_reg", "std": "std_reg"})

    coaches_df = pd.merge(
        coaches_regseason_win_df,
        coaches_postseason_win_df,
        on="CoachName",
        how="left"
    ).fillna(0)

    # Get coaches for the year and only grab the most recent coach for a certain team
    curr_coaches = (
        mens_data["MTeamCoaches"][
            mens_data["MTeamCoaches"]["Season"] == year
        ]
        .sort_values("FirstDayNum")
        .groupby("TeamID")["CoachName"]
        .last()
        .reset_index()
    )

    # Get coach stats for current coaches
    coaches_df = pd.merge(
        curr_coaches,
        coaches_df,
        on="CoachName",
        how="left"
    )


    feature_df = (
        pd.merge(
            ratings_df,
            coaches_df
        )
        .drop(["TeamName", "CoachName"], axis=1)
    )

    
    return feature_df


def merge_features_to_games(feature_df, post_season_df, year, training=True):
    
    post_season_merged = pd.merge(
        pd.merge(
            feature_df,
            post_season_df,
            on="TeamID",
        ),
        feature_df,
        left_on="OppID",
        right_on="TeamID",
        suffixes=("_team", "_opp")
    )
    if training:
        post_season_merged["win"] = post_season_merged["team_score"] > post_season_merged["opp_score"]
        post_season_merged = (
            post_season_merged
            .drop(
                ["TeamID_team", "team_score", "OppID", "TeamID_opp", "opp_score", "DayNum"], 
                axis=1
            )
        )
    return post_season_merged

In [ ]:
#TODO: Add features from box scores

In [5]:
def save_data(train_dict, _dir="baseline"):
    if not os.path.isdir(_dir):
        os.mkdir(_dir)
    for year in tqdm(train_dict):
        train_dict[year].to_csv(f"{_dir}/{year}.csv", index=False)
        
def load_data(_dir="baseline"):
    if not os.path.isdir(_dir):
        raise NotADirectoryError(f"{_dir} is not a directory")
    else:
        train_data = dict()
        for dirname, _, filenames in tqdm(os.walk(_dir)):
            for filename in filenames:
                table_name = filename.split('.')[0]
                table_path = os.path.join(dirname, filename)
                try:
                    train_data[table_name] = pd.read_csv(table_path, index_col=False)
                except UnicodeDecodeError:
                    train_data[table_name] = pd.read_csv(table_path, encoding='cp1252')
                except Exception as e:
                    print(f"Error with {filename}: {e}")
    return train_data

# Model V2
Improvements:
- Build in detailed stats from box scores
- Build in conference stats
- Tune hyperparams
- Add in tempo
- Add in experience (maybe)
- Maybe add round number as a feature

# Baseline Model

Use a simple rating system (SRS) combined with KenPom to fit a model predicting the probability of a given matchup in the tournament. Then randomly sample from the distribution outputed from the model to create multiple submissions.

### Features:
- SRS system from the regular season
- KenPom ranking system
- Coach historical success in post season and regular season

### Model: XGBoost


In [6]:
# This function will change a lot based on what we are trying to predict
# Simplest training method is to grab team ids from previous years and pull in reg season stats to make a prediction
# what we should try to get to is running simulations and making predictions based on matchups then have some sort of loss metric for how good or bad a bracket is.
# Also adding stats like if they're on a run or not would be cool (tough to do at inference time)
def create_mens_training_data():
    
    training_data = dict()
    
    for year in tqdm(range(2003, 2024)):
        
        feature_df = get_features(mens_data, year=year, systems=["POM", "NOL", "EBP"])
        post_season_df = get_post_season(mens_data, year)
        post_season_merged = merge_features_to_games(feature_df, post_season_df, year)
        
        training_data[year] = post_season_merged
    
    return training_data

In [7]:
# Need to join this on who is in the tournament every year and create predictions based on matchups 2003-2023
train_dict = create_mens_training_data()

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:48<00:00,  2.31s/it]


In [25]:
# save_data(train_dict, "../data/baseline")

In [26]:
# train_data=load_data("../data/baseline")

In [8]:
all_data = pd.concat(train_dict.values(), ignore_index=True)

In [9]:
train = all_data[all_data["Season"] != 2023]
test = all_data[all_data["Season"] == 2023]

X_train = train.drop("win", axis=1)
y_train = train["win"]
X_test = test.drop("win", axis=1)
y_test = test["win"]

training_cols = X_train.columns
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### XGBoost

In [10]:
model = xgboost.XGBClassifier(n_estimators=500, subsample=.9, early_stopping=True, gamma=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping=True,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [11]:
sorted({k: v for k, v in zip(training_cols, model.feature_importances_)}.items(), key=lambda x: -x[1])

[('rating_opp', 0.09714115),
 ('rating_team', 0.058258213),
 ('POM_rolling_opp', 0.054239083),
 ('count_post_team', 0.052308846),
 ('mean_post_opp', 0.03871247),
 ('NOL_rolling_opp', 0.038213816),
 ('NOL_latest_opp', 0.03778072),
 ('mean_post_team', 0.036736447),
 ('POM_rolling_team', 0.03625869),
 ('EBP_rolling_opp', 0.036231138),
 ('EBP_latest_opp', 0.03316768),
 ('std_reg_team', 0.032831904),
 ('std_reg_opp', 0.032565724),
 ('POM_latest_team', 0.03229817),
 ('EBP_rolling_team', 0.032254502),
 ('POM_latest_opp', 0.031666696),
 ('NOL_latest_team', 0.031514853),
 ('std_post_opp', 0.030288745),
 ('count_post_opp', 0.029899059),
 ('Season', 0.02981774),
 ('NOL_rolling_team', 0.0293188),
 ('count_reg_team', 0.029226286),
 ('mean_reg_opp', 0.028456144),
 ('mean_reg_team', 0.027997544),
 ('EBP_latest_team', 0.027695885),
 ('std_post_team', 0.027570788),
 ('count_reg_opp', 0.027549058)]

In [12]:
 #XGB
train_predicted_output = pd.DataFrame(
    {
        # "Predicted": model.predict_proba(X_test)[:, 1], 
        "Predicted": model.predict(X_train),
        "Actual": y_train.astype(int)
    }
)

predicted_output = pd.DataFrame(
    {
        # "Predicted": model.predict_proba(X_test)[:, 1], 
        "Predicted": model.predict(X_test),
        "Actual": y_test.astype(int)
    }
)

print(f"Train Accuracy: {np.mean(np.where(train_predicted_output['Predicted'] == train_predicted_output['Actual'], 1, 0))}")
print(f"Test Accuracy: {np.mean(np.where(predicted_output['Predicted'] == predicted_output['Actual'], 1, 0))}")

Train Accuracy: 0.9951884522854851
Test Accuracy: 0.6567164179104478


In [13]:
def create_testing_data(df, feature_df, seed_lookup, scaler, year):
    """
    matchup_df: pd.DataFrame
    scaler: StandardScaler
    year: int
    """
    
    matchup_df = df.copy()
    
    matchup_df["TeamID"] = matchup_df["StrongSeed"].apply(
        lambda x: seed_lookup[x]
    )
    matchup_df["OppID"] = matchup_df["WeakSeed"].apply(
        lambda x: seed_lookup[x]
    )
    
    post_season_merged = merge_features_to_games(feature_df, matchup_df, year, training=False)

    post_season_merged = post_season_merged[scaler.feature_names_in_]

    X_test = scaler.transform(post_season_merged)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    return X_test_tensor

def probabilistic_choice(row):
    return np.random.choice([row['StrongSeed'], row['WeakSeed']], p=[row['prob'], 1-row['prob']])

def get_model_predictions(model, inputs, matchup_df, model_type):
    """
    matchup_df: pd.DataFrame
    scaler: StandardScaler
    year: int
    """
    if model_type == "torch":
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Inference mode, gradients not needed
            outputs = model(inputs)
    elif model_type == "xgb":
        outputs = model.predict_proba(inputs)[:, 1]
    
    pred_df = matchup_df.copy()
    pred_df["prob"] = outputs
    pred_df["winner"] = pred_df.apply(probabilistic_choice, axis=1)

    return pred_df


def create_feature_df(year, systems=["POM", "NOL", "EBP"]):
    
    seeding_df = mens_data["MNCAATourneySeeds"][
        mens_data["MNCAATourneySeeds"]["Season"] == year
    ]

    feature_df = get_features(mens_data, year=year, systems=systems)
    
    res = pd.merge(seeding_df, feature_df, on="TeamID")

    return res

# Run Simulation Based on Model
Recursively loop through the seeding df to pick teams at each round.

TODO: Swap out function to use baseline model predictions

In [14]:
def create_submission(model, gender, bracket, year, systems=["POM", "NOL", "EBP"]):

    df = mens_data["MNCAATourneySlots"][
        (mens_data["MNCAATourneySlots"]["Season"] == year)
    ]
    seeding_df = mens_data["MNCAATourneySeeds"][
        mens_data["MNCAATourneySeeds"]["Season"] == year
    ]

    seed_lookup = {
        k: v for k, v in zip(
            seeding_df["Seed"], 
            seeding_df["TeamID"]
        )
    }

    feature_df = get_features(mens_data, year=year, systems=systems)

    def simulate(_df, round_num=0, results=None):
        
        df = _df.copy()
        
        if results is None:
            results = {}
            
        if round_num > 6:
            return results
        
        if round_num == 0: # Play-IN games
            temp_df = df[~df["Slot"].str.startswith("R")]
        
        else:
            temp_df = df[df["Slot"].str.startswith(f"R{round_num}")]
            temp_df["StrongSeed"] = temp_df["StrongSeed"].apply(
                lambda x: results[x] if x in results else x
            )
            temp_df["WeakSeed"] = temp_df["WeakSeed"].apply(
                lambda x: results[x] if x in results else x
            )
            

        inputs = create_testing_data(temp_df, feature_df, seed_lookup, scaler, year)
        temp_df = get_model_predictions(model, inputs, temp_df, model_type="xgb")

        for k, v in zip(temp_df["Slot"], temp_df["winner"]):
            results[k] = v
        
        results = simulate(df, round_num + 1, results)
        return results
    
    round_winner = simulate(df)
    
    df["Team"] = df["Slot"].apply(lambda x: round_winner[x])
    df["Tournament"] = gender
    df["Bracket"] = bracket
    
    return df[["Tournament", "Bracket", "Slot", "Team"]]

In [16]:
# Add multiprocessing to this function or offload to GPU
brackets = {}
for i in tqdm(range(100)):
    brackets[i] = create_submission(model=model, gender="M", bracket=i, year=2023)

100%|██████████| 100/100 [04:33<00:00,  2.74s/it]


In [31]:
def worker_function(i):
    return i, create_submission(model=model, gender="M", bracket=i, year=2023)


from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm as tq

# Assuming your create_submission function is defined elsewhere and accessible

def worker_function(i):
    return create_submission(model="YourModel", gender="M", bracket=i, year=2023)


# def run_parallel():
#     brackets = {}
#     pool_size = multiprocessing.cpu_count()  # Number of processes to create

#     with multiprocessing.Pool(pool_size) as pool:
#         # Map the range of inputs to the worker function
#         # The tqdm call is moved here to track progress of the parallel tasks
#         results = list(tqdm(pool.imap(worker_function, range(100)), total=100))

#     # Populate the brackets dictionary with the results
#     for i, result in results:
#         brackets[i] = result

#     return brackets

# if __name__ == "__main__":
#     # Using a smaller number of processes in Jupyter might be more stable
#     pool_size = min(multiprocessing.cpu_count(), 4)  
#     pool = multiprocessing.Pool(pool_size)
    
#     # Use a list to collect the results
#     results = []
#     for _ in tqdm(pool.imap_unordered(worker_function, range(100)), total=100):
#         results.append(_)
    
#     pool.close()
#     pool.join()

#     # Now process the results
#     brackets = {result['bracket']: result for result in results}

Process SpawnProcess-331:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/mm_2024/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/mm_2024/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/envs/mm_2024/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/opt/miniconda3/envs/mm_2024/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'worker_function' on <module '__main__' (built-in)>
Process SpawnProcess-332:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/mm_2024/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/mm_2024/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [17]:
df = pd.concat(brackets.values(), ignore_index=False)

In [18]:
def get_successors(seed):
    net = networkx.DiGraph()

    slot_df = mens_data["MNCAATourneySlots"][
        (mens_data["MNCAATourneySlots"]["Season"] == 2023)
    ]

    net.add_edges_from([i for i in zip(slot_df["WeakSeed"].values, slot_df["Slot"].values)])
    net.add_edges_from([i for i in zip(slot_df["StrongSeed"].values, slot_df["Slot"].values)])

    successors = [i[1] for i in dfs_edges(net, seed)]

    return successors

In [19]:
def check_bracket_distribution(df, seed, verbose=False):
    """
    df: pd.DataFrame
        This is the submission dataframe for all brackets
    team: str
        a seed representing the team
    """
    successors = get_successors(seed)
    path_df = df[df["Slot"].isin(successors)]
    path_df = path_df.groupby("Slot")["Team"].apply(lambda x: np.mean(x == seed))

    if verbose:
        seeding_teams = pd.merge(mens_data["MNCAATourneySeeds"][
            mens_data["MNCAATourneySeeds"]["Season"] == 2023
        ], mens_data["MTeams"], on="TeamID")

        try:
            team_name = seeding_teams[seeding_teams["Seed"] == seed]["TeamName"].values[0]
        except:
            team_name=""

        print("*"*50)
        print(f"{team_name} CHANCE TO WIN IT ALL: {path_df[-1]}")
        print("*"*50)

    return path_df

In [20]:
tournament_2023 = pd.merge(
    mens_data["MNCAATourneySeeds"][
        mens_data["MNCAATourneySeeds"]["Season"] == 2023
    ], 
    mens_data["MTeams"], 
    on="TeamID"
)

In [21]:
tournament_2023["round_win_prob"]= tournament_2023["Seed"].apply(lambda x: check_bracket_distribution(df, x)[:6].values)

tournament_2023[
    [
        "round_of_32",
        "sweet16",
        "elite8",
        "final4",
        "final",
        "champ",
    ]
] = list(tournament_2023["round_win_prob"])

In [22]:
tournament_2023.sort_values(by=["champ", "final", "final4", "elite8", "sweet16", "round_of_32"], ascending=False)

,Season,Seed,TeamID,TeamName,FirstD1Season,LastD1Season,round_win_prob,round_of_32,sweet16,elite8,final4,final,champ
0,2023,W01,1345,Purdue,1985,2024,"[0.99, 0.95, 0.85, 0.68, 0.42, 0.22]",0.99,0.95,0.85,0.68,0.42,0.22
17,2023,X01,1104,Alabama,1985,2024,"[0.92, 0.73, 0.48, 0.36, 0.19, 0.16]",0.92,0.73,0.48,0.36,0.19,0.16
56,2023,Z06,1395,TCU,1985,2024,"[0.54, 0.4, 0.28, 0.17, 0.13, 0.07]",0.54,0.40,0.28,0.17,0.13,0.07
53,2023,Z03,1211,Gonzaga,1985,2024,"[0.81, 0.19, 0.15, 0.09, 0.08, 0.07]",0.81,0.19,0.15,0.09,0.08,0.07
43,2023,Y10,1336,Penn St,1985,2024,"[0.68, 0.46, 0.16, 0.12, 0.07, 0.04]",0.68,0.46,0.16,0.12,0.07,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2023,W12,1331,Oral Roberts,1985,2024,"[0.06, 0.01, 0.0, 0.0, 0.0, 0.0]",0.06,0.01,0.00,0.00,0.00,0.00
8,2023,W09,1194,FL Atlantic,1994,2024,"[0.09, 0.0, 0.0, 0.0, 0.0, 0.0]",0.09,0.00,0.00,0.00,0.00,0.00
16,2023,W16b,1411,TX Southern,1985,2024,"[0.01, 0.0, 0.0, 0.0, 0.0, 0.0]",0.01,0.00,0.00,0.00,0.00,0.00
15,2023,W16a,1192,F Dickinson,1985,2024,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.00,0.00,0.00,0.00,0.00,0.00
